In [2]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.regularizers import L1L2
from math import sqrt
import matplotlib
# be able to save images on server
matplotlib.use('Agg')
from matplotlib import pyplot
import numpy
 
# date-time parsing function for loading the dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')
 
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	return df
 
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)
 
# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]
 
# scale train and test data to [-1, 1]
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled
 
# inverse scaling for a forecasted value
def invert_scale(scaler, X, yhat):
	new_row = [x for x in X] + [yhat]
	array = numpy.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]
 
# fit an LSTM network to training data
def fit_lstm(train, n_batch, nb_epoch, n_neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
		model.reset_states()
	return model
 
# run a repeated experiment
def experiment(series, n_lag, n_repeats, n_epochs, n_batch, n_neurons):
	# transform data to be stationary
	raw_values = series.values
	diff_values = difference(raw_values, 1)
	# transform data to be supervised learning
	supervised = timeseries_to_supervised(diff_values, n_lag)
	supervised_values = supervised.values[n_lag:,:]
	# split data into train and test-sets
	train, test = supervised_values[0:-12], supervised_values[-12:]
	# transform the scale of the data
	scaler, train_scaled, test_scaled = scale(train, test)
	# run experiment
	error_scores = list()
	for r in range(n_repeats):
		# fit the model
		train_trimmed = train_scaled[2:, :]
		lstm_model = fit_lstm(train_trimmed, n_batch, n_epochs, n_neurons)
		# forecast test dataset
		test_reshaped = test_scaled[:,0:-1]
		test_reshaped = test_reshaped.reshape(len(test_reshaped), 1, 1)
		output = lstm_model.predict(test_reshaped, batch_size=n_batch)
		predictions = list()
		for i in range(len(output)):
			yhat = output[i,0]
			X = test_scaled[i, 0:-1]
			# invert scaling
			yhat = invert_scale(scaler, X, yhat)
			# invert differencing
			yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
			# store forecast
			predictions.append(yhat)
		# report performance
		rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
		print('%d) Test RMSE: %.3f' % (r+1, rmse))
		error_scores.append(rmse)
	return error_scores
 
# configure the experiment
def run():
	# load dataset
	series = read_csv('sales-of-shampoo-over-a-three-ye.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
	# configure the experiment
	n_lag = 1
	n_repeats = 30
	n_epochs = 1000
	n_batch = 4
	n_neurons = 3
	# run the experiment
	results = DataFrame()
	results['results'] = experiment(series, n_lag, n_repeats, n_epochs, n_batch, n_neurons)
	# summarize results
	print(results.describe())
	# save boxplot
	results.boxplot()
	pyplot.savefig('experiment_baseline.png')
 
# entry point
run()

1) Test RMSE: 103.801
2) Test RMSE: 102.154
3) Test RMSE: 97.784
4) Test RMSE: 97.424
5) Test RMSE: 96.103
6) Test RMSE: 101.939
7) Test RMSE: 107.642
8) Test RMSE: 105.947
9) Test RMSE: 100.953
10) Test RMSE: 98.215
11) Test RMSE: 100.214
12) Test RMSE: 105.046
13) Test RMSE: 98.000
14) Test RMSE: 104.419
15) Test RMSE: 91.081
16) Test RMSE: 104.118
17) Test RMSE: 89.037
18) Test RMSE: 105.725
19) Test RMSE: 101.077
20) Test RMSE: 100.474
21) Test RMSE: 100.627
22) Test RMSE: 103.769
23) Test RMSE: 100.732
24) Test RMSE: 105.169
25) Test RMSE: 99.386
26) Test RMSE: 90.229
27) Test RMSE: 90.224
28) Test RMSE: 93.399
29) Test RMSE: 106.395
30) Test RMSE: 94.516
          results
count   30.000000
mean    99.853247
std      5.234192
min     89.036614
25%     97.514189
50%    100.679157
75%    104.038815
max    107.641611


In [5]:
# fit an LSTM network to training data
def fit_lstm(train, n_batch, nb_epoch, n_neurons, reg):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True, bias_regularizer=reg))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
		model.reset_states()
	return model
 
# run a repeated experiment
def experiment(series, n_lag, n_repeats, n_epochs, n_batch, n_neurons, reg):
	# transform data to be stationary
	raw_values = series.values
	diff_values = difference(raw_values, 1)
	# transform data to be supervised learning
	supervised = timeseries_to_supervised(diff_values, n_lag)
	supervised_values = supervised.values[n_lag:,:]
	# split data into train and test-sets
	train, test = supervised_values[0:-12], supervised_values[-12:]
	# transform the scale of the data
	scaler, train_scaled, test_scaled = scale(train, test)
	# run experiment
	error_scores = list()
	for r in range(n_repeats):
		# fit the model
		train_trimmed = train_scaled[2:, :]
		lstm_model = fit_lstm(train_trimmed, n_batch, n_epochs, n_neurons, reg)
		# forecast test dataset
		test_reshaped = test_scaled[:,0:-1]
		test_reshaped = test_reshaped.reshape(len(test_reshaped), 1, 1)
		output = lstm_model.predict(test_reshaped, batch_size=n_batch)
		predictions = list()
		for i in range(len(output)):
			yhat = output[i,0]
			X = test_scaled[i, 0:-1]
			# invert scaling
			yhat = invert_scale(scaler, X, yhat)
			# invert differencing
			yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
			# store forecast
			predictions.append(yhat)
		# report performance
		rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
		print('%d) Test RMSE: %.3f' % (r+1, rmse))
		error_scores.append(rmse)
	return error_scores
 
# configure the experiment
# def run():
# load dataset
series = read_csv('sales-of-shampoo-over-a-three-ye.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
# configure the experiment
n_lag = 1
n_repeats = 30
n_epochs = 1000
n_batch = 4
n_neurons = 3
regularizers = [L1L2(l1=0.0, l2=0.0), L1L2(l1=0.01, l2=0.0), L1L2(l1=0.0, l2=0.01), L1L2(l1=0.01, l2=0.01)]
# run the experiment
results = DataFrame()
for reg in regularizers:
    name = ('l1 %.2f,l2 %.2f' % (reg.l1, reg.l2))
    results[name] = experiment(series, n_lag, n_repeats, n_epochs, n_batch, n_neurons, reg)
# summarize results
print(results.describe())
# save boxplot
results.boxplot()
pyplot.savefig('experiment_reg_bias.png')

1) Test RMSE: 102.756
2) Test RMSE: 103.659
3) Test RMSE: 101.663
4) Test RMSE: 96.790
5) Test RMSE: 90.144
6) Test RMSE: 98.907
7) Test RMSE: 103.628
8) Test RMSE: 99.703
9) Test RMSE: 95.826
10) Test RMSE: 100.191
11) Test RMSE: 96.548
12) Test RMSE: 91.192
13) Test RMSE: 95.299
14) Test RMSE: 93.010
15) Test RMSE: 102.276
16) Test RMSE: 90.415
17) Test RMSE: 109.124
18) Test RMSE: 99.815
19) Test RMSE: 92.226
20) Test RMSE: 96.549
21) Test RMSE: 97.072
22) Test RMSE: 101.863
23) Test RMSE: 97.426
24) Test RMSE: 93.314
25) Test RMSE: 102.036
26) Test RMSE: 100.225
27) Test RMSE: 100.881
28) Test RMSE: 86.669
29) Test RMSE: 98.511
30) Test RMSE: 96.206
1) Test RMSE: 98.494
2) Test RMSE: 97.998
3) Test RMSE: 100.179
4) Test RMSE: 107.570
5) Test RMSE: 100.809
6) Test RMSE: 96.645
7) Test RMSE: 93.153
8) Test RMSE: 99.547
9) Test RMSE: 112.056
10) Test RMSE: 110.923
11) Test RMSE: 104.424
12) Test RMSE: 103.315
13) Test RMSE: 100.688
14) Test RMSE: 95.311
15) Test RMSE: 103.196
16) Test

In [6]:
def fit_lstm(train, n_batch, nb_epoch, n_neurons, reg):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True, kernel_regularizer=reg))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
        model.reset_states()
    return model
 
# run a repeated experiment
def experiment(series, n_lag, n_repeats, n_epochs, n_batch, n_neurons, reg):
    # transform data to be stationary
    raw_values = series.values
    diff_values = difference(raw_values, 1)
    # transform data to be supervised learning
    supervised = timeseries_to_supervised(diff_values, n_lag)
    supervised_values = supervised.values[n_lag:,:]
    # split data into train and test-sets
    train, test = supervised_values[0:-12], supervised_values[-12:]
    # transform the scale of the data
    scaler, train_scaled, test_scaled = scale(train, test)
    # run experiment
    error_scores = list()
    for r in range(n_repeats):
        # fit the model
        train_trimmed = train_scaled[2:, :]
        lstm_model = fit_lstm(train_trimmed, n_batch, n_epochs, n_neurons, reg)
        # forecast test dataset
        test_reshaped = test_scaled[:,0:-1]
        test_reshaped = test_reshaped.reshape(len(test_reshaped), 1, 1)
        output = lstm_model.predict(test_reshaped, batch_size=n_batch)
        predictions = list()
        for i in range(len(output)):
            yhat = output[i,0]
            X = test_scaled[i, 0:-1]
            # invert scaling
            yhat = invert_scale(scaler, X, yhat)
            # invert differencing
            yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
            # store forecast
            predictions.append(yhat)
        # report performance
        rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
        print('%d) Test RMSE: %.3f' % (r+1, rmse))
        error_scores.append(rmse)
    return error_scores
 
# configure the experiment
# def run():
# load dataset
series = read_csv('sales-of-shampoo-over-a-three-ye.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
# configure the experiment
n_lag = 1
n_repeats = 30
n_epochs = 1000
n_batch = 4
n_neurons = 3
regularizers = [L1L2(l1=0.0, l2=0.0), L1L2(l1=0.01, l2=0.0), L1L2(l1=0.0, l2=0.01), L1L2(l1=0.01, l2=0.01)]
# run the experiment
results = DataFrame()
for reg in regularizers:
    name = ('l1 %.2f,l2 %.2f' % (reg.l1, reg.l2))
    results[name] = experiment(series, n_lag, n_repeats, n_epochs, n_batch, n_neurons, reg)
# summarize results
print(results.describe())
# save boxplot
results.boxplot()
pyplot.savefig('experiment_reg_input.png')

1) Test RMSE: 101.202
2) Test RMSE: 90.671
3) Test RMSE: 101.603
4) Test RMSE: 102.153
5) Test RMSE: 89.749
6) Test RMSE: 98.850
7) Test RMSE: 102.873
8) Test RMSE: 94.765
9) Test RMSE: 91.590
10) Test RMSE: 89.169
11) Test RMSE: 98.438
12) Test RMSE: 102.296
13) Test RMSE: 109.811
14) Test RMSE: 97.078
15) Test RMSE: 101.633
16) Test RMSE: 94.771
17) Test RMSE: 95.600
18) Test RMSE: 90.956
19) Test RMSE: 97.566
20) Test RMSE: 99.801
21) Test RMSE: 100.491
22) Test RMSE: 93.169
23) Test RMSE: 101.537
24) Test RMSE: 96.519
25) Test RMSE: 106.435
26) Test RMSE: 104.144
27) Test RMSE: 109.896
28) Test RMSE: 105.345
29) Test RMSE: 93.409
30) Test RMSE: 92.402
1) Test RMSE: 111.682
2) Test RMSE: 119.877
3) Test RMSE: 113.314
4) Test RMSE: 102.117
5) Test RMSE: 114.740
6) Test RMSE: 108.843
7) Test RMSE: 119.425
8) Test RMSE: 121.916
9) Test RMSE: 104.127
10) Test RMSE: 105.037
11) Test RMSE: 100.552
12) Test RMSE: 107.203
13) Test RMSE: 119.901
14) Test RMSE: 121.888
15) Test RMSE: 110.017


KeyboardInterrupt: 

In [ ]:
def fit_lstm(train, n_batch, nb_epoch, n_neurons, reg):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True, recurrent_regularizer=reg))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
        model.reset_states()
    return model
 
# run a repeated experiment
def experiment(series, n_lag, n_repeats, n_epochs, n_batch, n_neurons, reg):
    # transform data to be stationary
    raw_values = series.values
    diff_values = difference(raw_values, 1)
    # transform data to be supervised learning
    supervised = timeseries_to_supervised(diff_values, n_lag)
    supervised_values = supervised.values[n_lag:,:]
    # split data into train and test-sets
    train, test = supervised_values[0:-12], supervised_values[-12:]
    # transform the scale of the data
    scaler, train_scaled, test_scaled = scale(train, test)
    # run experiment
    error_scores = list()
    for r in range(n_repeats):
        # fit the model
        train_trimmed = train_scaled[2:, :]
        lstm_model = fit_lstm(train_trimmed, n_batch, n_epochs, n_neurons, reg)
        # forecast test dataset
        test_reshaped = test_scaled[:,0:-1]
        test_reshaped = test_reshaped.reshape(len(test_reshaped), 1, 1)
        output = lstm_model.predict(test_reshaped, batch_size=n_batch)
        predictions = list()
        for i in range(len(output)):
            yhat = output[i,0]
            X = test_scaled[i, 0:-1]
            # invert scaling
            yhat = invert_scale(scaler, X, yhat)
            # invert differencing
            yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
            # store forecast
            predictions.append(yhat)
        # report performance
        rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
        print('%d) Test RMSE: %.3f' % (r+1, rmse))
        error_scores.append(rmse)
    return error_scores
 
# configure the experiment
# def run():
# load dataset
series = read_csv('sales-of-shampoo-over-a-three-ye.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
# configure the experiment
n_lag = 1
n_repeats = 30
n_epochs = 1000
n_batch = 4
n_neurons = 3
regularizers = [L1L2(l1=0.0, l2=0.0), L1L2(l1=0.01, l2=0.0), L1L2(l1=0.0, l2=0.01), L1L2(l1=0.01, l2=0.01)]
# run the experiment
results = DataFrame()
for reg in regularizers:
    name = ('l1 %.2f,l2 %.2f' % (reg.l1, reg.l2))
    results[name] = experiment(series, n_lag, n_repeats, n_epochs, n_batch, n_neurons, reg)
# summarize results
print(results.describe())
# save boxplot
results.boxplot()
pyplot.savefig('experiment_reg_recurrent.png')

1) Test RMSE: 93.885
